In [40]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print (tf.__version__)
print (sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print (module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [41]:
#普通function转换为tf的API

# tf.function and autograph(tf.function 依赖的机制)
def scaled_elu(z, scale=1.0, alpha=1.0):
    #z >= 0 ? scale *z:scale*alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale* tf.where(is_positive, z, alpha*z)

# 使用tf相关函数来操作，可以做到优化？用纯python不太好用

print (scaled_elu(tf.constant(-3.0)))
print (scaled_elu(tf.constant([-3.0,-2.5])))


# 第一种方式转换为tf.funciton 
scaled_elu_tf = tf.function(scaled_elu)
print (scale_elu_tf(tf.constant(-3.0)))
print (scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-3.0, shape=(), dtype=float32)
tf.Tensor([-3.  -2.5], shape=(2,), dtype=float32)
tf.Tensor(-3.0, shape=(), dtype=float32)
True


In [7]:
'''转化成tf.function 之后会被优化，执行结果变快了'''

%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))

16.6 ms ± 637 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.4 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
def scaled_elu_python(z, scale=1.0, alpha=1.0):
    #z >= 0 ? scale *z:scale*alpha * tf.nn.elu(z)
    is_positive = np.greater_equal(z, 0.0)
    return scale* np.where(is_positive, z, alpha*z)

In [14]:
print (scaled_elu_python(tf.constant(-3.0)))
print (scaled_elu_python(tf.constant([-3.0,-2.5])))

-3.0
[-3.  -2.5]


In [53]:
# 1 +1/2 + 1/2^2+ ... + 

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /=2.0
    return total

print (converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [54]:
#转换为tf的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [55]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  is_positive = ag__.converted_call('greater_equal', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (z, 0.0), None)
  do_return = True
  retval_ = scale * ag__.converted_call('where', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (is_positive, z, alpha * z), None)
  cond = ag__.is_undefined_return(retval_)

  def get_state():
    return ()

  def set_state(_):
    pass

  def if_true():
    retval_ = None
    return retval_

  def if_false():
    return retval_
  retval_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state)
  return retval_

```

In [58]:
display_tf_code(converge_to_2)

ConversionError: converting <tensorflow.python.eager.def_function.Function object at 0x132afbb00>: AttributeError: 'Function' object has no attribute '__code__'

In [23]:
#神经网络中多用variable，需要对variable变量初始化

var = tf.Variable(0.)
@tf.function
def add_21():
    return var.assign_add(21)

print (add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [77]:
#可不可以对输入类型做限定？ 可以input_signature

@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

try:
    print (cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print (ex)
print (cube(tf.constant([1, 2, 3])))
print (cube)

Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=16970, shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name='x'),))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [78]:
#  @tf.function py func -> graph
# get_concrete_function -> add input signature -> 变成可以保存的图结构  SavedModel

cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32))

print (cube_func_int32)
cube_func_int32(tf.constant([1, 2, 3]))

<tf.Tensor: id=16983, shape=(3,), dtype=int32, numpy=array([ 1,  8, 27], dtype=int32)>

In [79]:
print (cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32 )))
print (cube_func_int32 is cube.get_concrete_function(
        tf.constant([1,2,3])))

True
True


In [80]:
cube_func_int32.graph

In [82]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [84]:
pow_op = cube_func_int32.graph.get_operations()[2]
print (pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [85]:
print(list(pow_op.inputs))
print (list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [86]:
cube_func_int32.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [88]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [89]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 55
}